# Setup

In [ ]:
%pip install kornia==0.4.0
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math 
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import kornia as K
import time

## Mount drive

In [ ]:
your_path = ''
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/'+your_path
import sys
sys.path.append(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## GPU

In [ ]:
print(torch.__version__)
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  DEVICE = 'cpu'
  print('CUDA is not available.  Training on CPU ...')
else:
  DEVICE = 'cuda'
  print('CUDA is available!  Training on GPU ...')



1.6.0
CUDA is available!  Training on GPU ...


# DATA

## Define transformatiom

In [ ]:
from datasets.utils.transforms import ColorJitter, ToGray, Rescale,ToTensor

## Dataset model

In [ ]:
from datasets.COCO.COCO_model import COCO_dataset



## Generate dataset and dataloader

In [ ]:
transform = transforms.Compose([Rescale((240,320)),
                                ColorJitter(), 
                                ToGray(),
                                ToTensor()]) 

dataset = COCO_dataset(path+'/datasets/COCO/labeled_coco.csv',
                                  path+'/datasets/COCO/val2017/', 
                                  transform=transform, 
                                  landmark_bool=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)

print(len(dataset))
print(len(dataloader))
for iter, (im,label) in enumerate(dataloader):
  print('ok')
  if iter>5:
    break


  

5000
1000
ok
ok
ok
ok
ok
ok
ok


## Test model

### plot function

In [ ]:
sys.path.append(path)
from utils.plot import plot_imgs


### test

In [ ]:
from utils.points import cords_to_map

for iter, (im, label) in enumerate(dataloader):
  label = label.type(torch.double)
  label = label.unsqueeze(1)
  imgs= K.tensor_to_image(im)
  size = im.size()
  map = cords_to_map(label, size, device=False)
  im_map = K.tensor_to_image(map)
  if iter%10==0:
    print('iteration {}/{} is running'.format(iter,len(dataloader)))
  plot_imgs(imgs, label=label)
  plot_imgs(im_map)
  if iter>2:
    break


Output hidden; open in https://colab.research.google.com to view.

# Net architecture

## Superpoint model

In [ ]:
from models.superpoint import SuperPointNet

## functions for Loss calculations

In [ ]:
from models.utils import detector_loss


# Training magic point on COCO

In [ ]:
import torch
import kornia as K
import torch.nn.functional as F


def detector_loss(true_map, chi, v_mask=None):
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  n, c, h, w = true_map.size()
  block_size = 8
  true_map = true_map.type(torch.float)
  unfolded_map = torch.nn.functional.unfold(true_map, block_size, stride=block_size)
  unfolded_map = unfolded_map.view(n, c * block_size ** 2, h // block_size, w // block_size)
  unfolded_map = unfolded_map.permute(0,2,3,1)
  shape = torch.cat([torch.tensor(unfolded_map.size())[:3], torch.tensor([1])], dim=0)
  unfolded_map = torch.cat([2*unfolded_map, torch.ones(tuple(shape)).to(DEVICE)], dim=3)
  noise = torch.rand(unfolded_map.size())*0.1
  noise = noise.to(DEVICE)
  label = torch.argmax(unfolded_map+noise,dim=3)
  #define valid mask
  if v_mask:
    valid_mask = v_mask.type(torch.float32).to(DEVICE)
  else:
    valid_mask = torch.ones_like(true_map, dtype=torch.float32).to(DEVICE)  
  # adjust valid_mask
  valid_mask = F.unfold(valid_mask, block_size, stride=block_size)
  valid_mask = valid_mask.view(n, c * block_size ** 2, h // block_size, w // block_size)
  valid_mask = valid_mask.permute(0,2,3,1)
  valid_mask = torch.prod(valid_mask, dim=3)
  label[valid_mask==0] = 65
  #get loss
  m = torch.nn.LogSoftmax(dim=1)  
  loss = torch.nn.NLLLoss(ignore_index=65)
  output = loss(m(chi), label)
  return output




## Train function

In [ ]:
def train_coco_magic(dataloader,writer, net, save_path, filename, lr=0.001):
  t_0 = time.time()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  net.to(DEVICE)
  model.train()
  for e in range(12):
    for iter, (im, label) in enumerate(dataloader):
      optimizer.zero_grad()
      im = im.to(DEVICE).type(torch.float)
      label = label.to(DEVICE)
      #go through model
      chi_points, desc = net(im)
      #get map label
      label = label.type(torch.double)
      size = im.size()
      map = cords_to_map(label, size)
      map[map<0.01] = 0
      #loss
      loss = detector_loss(map, chi_points)
      loss.backward()
      optimizer.step()
      writer.add_scalar("Loss/train", loss, e*len(dataloader)+iter)
      if iter%10==0:
        print('iteration {}/{} is running'.format(e*len(dataloader)+iter,12*len(dataloader)))
        print('loss is:',loss.item())
      if iter%50==0:
        t_c = time.time()
        minute = (t_c-t_0)/60
        print('saving weights from iteration {} with loss {}, {} minutes pased'.format(e*len(dataloader)+iter,loss.item(),int(minute)))
        torch.save(model.state_dict(), save_path+filename)
  # Save weights
  torch.save(model.state_dict(), save_path+filename)
  t_f = time.time()
  hours = (t_f-t_0)/3600
  print('finished in {} hours'.format(hours))

## Run

In [ ]:
#tensorflow
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os 

%load_ext tensorboard

logs_base_dir = 'logs'
os.makedirs(logs_base_dir, exist_ok=True)

log_dir_magic_synthetic_loss = "%s/magicpoint/coco_loss/%s" % (logs_base_dir,  datetime.now().strftime("%m%d-%H%M"))
writer_magic = SummaryWriter(log_dir_magic_synthetic_loss)


dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)
model = SuperPointNet(superpoint_bool=False)
weights_path = path+'/weights/magic_coco_weights_iter2.pth'
model.load_state_dict(torch.load(weights_path,
                               map_location=lambda storage, loc: storage))



train_coco_magic(dataloader, writer_magic, model,path, '/weights/magic_coco_weights_test.pth')


In [ ]:
#writer_magic.flush()
%tensorboard --logdir 'logs'